In [ ]:
# Step 1: Install Necessary Libraries
!pip install -q pandas requests
#!pip uninstall markdown -y
!pip install markdown


In [ ]:
# Step 2: Import Libraries
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import markdown
import os


In [ ]:
import os
env_vars = !cat .env
for var in env_vars:
    key, value = var.split('=')
    print(key)
    #print(value)
    os.environ[key] = value

In [ ]:
# Step 3: Load Data
# Read report.txt which is a markdown file
with open('report.txt', 'r') as file:
    report_content = file.read()

# Step 4: Load SEO Keywords
# Read SEO keywords from 'seo_report.md'
with open('seo_report.md', 'r') as file:
    seo_content = file.read()

In [ ]:
# Extract keywords assuming they are listed after 'SEO Keywords:'
import re
keywords_match = re.search(r'SEO Keywords:\s*(.*)', seo_content)
if keywords_match:
    seo_keywords = keywords_match.group(1)
else:
    seo_keywords = ""

print(seo_keywords)


In [ ]:
# Step 5: Load CSV Data
# Read output.csv which contains the recommendations in a tabular format
output_data = pd.read_csv('output.csv')

# Convert the CSV data to an HTML table
output_html_table = output_data.to_html(index=False)
print(output_html_table)


In [ ]:
# Step 6: Process Data
# Add SEO keywords to the post content
#report_content += f"\n\n#### SEO Keywords:\n{seo_keywords}"

# Convert Markdown to HTML
post_content_html = markdown.markdown(report_content)

# Define CSS styles for the table
table_styles = """
<style>
    .forex-table {
        width: 100%;
        border-collapse: collapse;
        margin-top: 20px;
    }
    .forex-table th, .forex-table td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: left;
    }
    .forex-table th {
        background-color: #f2f2f2;
    }
    .forex-table tr:nth-child(even) {
        background-color: #f9f9f9;
    }
</style>
"""

# Append the styled HTML table to the post content
# Ensure your HTML table string (`output_html_table`) has the class 'forex-table'
# You might need to adjust how you generate or modify `output_html_table` to include this class
post_content_html += f"{table_styles}<h3>Forex Trade Recommendations Table</h3>\n<table class='forex-table'>{output_html_table}</table>"

In [ ]:
def get_tag_ids(tag_names):
    # This function should query your WordPress for tag names and return their IDs.
    # For simplicity, let's assume it returns a static mapping for the example tag names.
    # In a real scenario, you'd query the WordPress API or have a pre-defined mapping.
    tag_name_to_id = {
        'forex': 24,  # Example ID
        'trade recommendations': 28,  # Example ID
        'forex market analysis': 27,  # Example ID
        'recommendations': 26
    }
    return [tag_name_to_id[tag] for tag in tag_names if tag in tag_name_to_id]


In [ ]:
import random
# Step 7: Generate WordPress Post
# Define your WordPress site and API credentials
wordpress_site = "https://" + str(os.getenv('WORDPRESS_URL'))
api_endpoint = f"{wordpress_site}/wp-json/wp/v2/posts"
username = 'XXXXXXXXXXX' #os.getenv('WORDPRESS_USERNAME')
#sprint(username)
password = 'XXXXXXXXXXX' #os.getenv('WORDPRESS_PASSWORD')
#print(password)

# Category ID (replace with your actual category ID)
category_id = 17
image_list = ["197","195","196", "197", "198", "199", "201", "202", "203", "204", "205", "237", "238", "239" , "235"]
random_image_list = random.choice(image_list)
# Get the tag IDs
tag_names = ['forex', 'forex market analysis', 'trade recommendations']
tag_ids = get_tag_ids(tag_names)
print(tag_ids)

# Open report.txt and read the first line to use as the title
with open('report.txt', 'r') as file:
    title = file.readline().strip().replace('#', '') # .strip() removes any leading/trailing whitespace or newline characters

# Create the post data
post_data = {
    'title': title,
    'content': post_content_html,
    'status': 'draft',  # 'publish'or 'draft' if you want to review before publishing
    'featured_media': random_image_list,
    'categories': [category_id],
    'tags': tag_ids,  # Use the list of tag IDs
}



In [ ]:
# Make the API request
response = requests.post(api_endpoint, json=post_data, auth=HTTPBasicAuth(username, password))

# Check the response
if response.status_code == 201:
    print("Post created successfully!")
else:
    print("Failed to create post:", response.status_code, response.text)